##一张图片，一个视频，一键换脸！

[![打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dream80/roop_colab/blob/main/roop_v1.3.1.ipynb)

基于[roop](https://github.com/s0md3v/roop)项目的一个脚本你，可以进行图片换脸，也可以视频换脸。可以自定义参数。  

特点：就是快捷方便，没有其他。  
更新：NSFW封印已经解除，剔除了没什么鸟用的但是很烦人的TF。

如果要高质量换脸还是推荐DeepFaclab[教程](https://www.deepfaker.xyz/)

脚本仅供学习研究，请勿用于任何非法用途！

使用教程：https://www.tonyisstark.com/1240.html  
离线版本：https://www.tonyisstark.com/1311.html

公众号：托尼不是塔克  
博客：[tonyisstark.com](https://www.tonyisstark.com/)

In [2]:
#@title 1.克隆源代码
!git clone https://github.com/dream80/roop_colab.git


Cloning into 'roop_colab'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 179 (delta 90), reused 80 (delta 54), pack-reused 39 (from 1)
Receiving objects: 100% (179/179), 15.07 MiB | 48.54 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [3]:
#@title 2.进入项目
%cd /content/roop_colab/roop/

/content/roop_colab/roop


In [4]:
#@title 3.安装依赖
#有红色警告不用担心！
%cd /content/roop_colab/roop/

!pip install -r requirements.txt
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install onnxruntime-gpu==1.22.0
# install system packages
!sudo apt-get update -y
!sudo apt-get install -y python3-tk tk

# install Python packages
!pip install customtkinter==5.2.0 tkinterdnd2==0.3.0
!pip install insightface


/content/roop_colab/roop
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 136.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likel

In [5]:
#@title 4.准备素材
source = "/content/roop_colab/abc.jpg" #@param {type:"string"}
target = "/content/roop_colab/bbbb.mp4" #@param {type:"string"}
output = "/content/roop_colab/out1.mp4" #@param {type:"string"}

source : 源素素材路径，包含人脸的图片  

target : 目标图片或者视频路径，也要包含人脸  

output : 输出路径，如果targe是图片，这里也改成图片后缀  


In [6]:
#@title 5.开始换脸

%cd /content/roop_colab/roop/

Device = "cuda" #@param ["cuda", "cpu"]

Processor = "face_swapper" #@param ["face_swapper face_enhancer", "face_swapper","face_enhancer"]

VideoEncoder = "libx264" #@param ['libx264', 'libx265', 'libvpx-vp9', 'h264_nvenc', 'hevc_nvenc']

VideoQuality = "35" #@param {type:"string"}

TempFormat = "jpg" #@param ["jpg", "png"]

TempQuality = "0" #@param {type:"string"}


KeepFPS = True #@param {type:"boolean"}
SkipAudio = False #@param {type:"boolean"}
KeepFrames = False #@param {type:"boolean"}
ManyFaces = False #@param {type:"boolean"}


KeepFPS ="--keep-fps" if KeepFPS==True else ""
SkipAudio ="--skip-audio" if SkipAudio==True else ""
KeepFrames ="--keep-frames" if KeepFrames==True else ""
ManyFaces ="--many-faces" if ManyFaces==True else ""



cmd = f"run.py --execution-provider {Device} -s {source} -t {target} -o {output} --frame-processor {Processor} --output-video-encoder {VideoEncoder} --output-video-quality {VideoQuality} {KeepFPS} {SkipAudio} {KeepFrames} {ManyFaces} --temp-frame-format {TempFormat} --temp-frame-quality {TempQuality}"
print("cmd:"+cmd)


# 创建models目录（如果不存在）
!mkdir -p models

# 下载inswapper_128.onnx（如果不存在）
!if [ ! -f "models/inswapper_128.onnx" ]; then \
  echo "Downloading inswapper_128.onnx..."; \
  wget https://github.com/dream80/roop_colab/releases/download/v0.0.1/inswapper_128.onnx && \
  mv inswapper_128.onnx models/; \
else \
  echo "inswapper_128.onnx already exists, skipping download."; \
fi

# 下载GFPGANv1.4.pth（如果不存在）
!if [ ! -f "models/GFPGANv1.4.pth" ]; then \
  echo "Downloading GFPGANv1.4.pth..."; \
  wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth && \
  mv GFPGANv1.4.pth models/; \
else \
  echo "GFPGANv1.4.pth already exists, skipping download."; \
fi

!python $cmd


/content/roop_colab/roop
cmd:run.py --execution-provider cuda -s /content/roop_colab/abc.jpg -t /content/roop_colab/bbbb.mp4 -o /content/roop_colab/out1.mp4 --frame-processor face_swapper --output-video-encoder libx264 --output-video-quality 35 --keep-fps    --temp-frame-format jpg --temp-frame-quality 0
--2025-10-07 10:59:26--  https://github.com/dream80/roop_colab/releases/download/v0.0.1/inswapper_128.onnx
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/647588832/cedb2105-b3b3-4944-ad18-e639d3334dae?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-07T11%3A48%3A21Z&rscd=attachment%3B+filename%3Dinswapper_128.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-07T10%3A48%3A04Z&ske=2025-10-07T11%3A48%3A

Device : 选CUDA就是使用GPU，选CPU就用CPU，如果你GPU配额用完了，可以用CPU顶一下。或者开Pro  

Precessor : 选择换脸还是人脸修复，默认全部启用，也可以单独使用一个功能。  

VideoEncoder : 视频编码器选择，一般用libx264。  

VideoQuality : 视频质量，默认35，取值范围0-100  

TempFormat ： 缓存帧格式，可以使jpg或者png  

TempQuality ： 视频质量，默认0，取值范围0-100  

KeepFPS : 保持帧率不变  

SkipAudio ： 丢弃音轨   

KeepFrames ： 保留分帧后的图片  

ManyFaces ： 替换所有人脸

#补充说明

  



## 视频文件在哪里？  

换脸成功后的视频保存在roop_colab文件里面  


## 如何使用自己的素材？  

如果要替换图片和视频，只需要在左边的文件管理里面右键上传，然后改一下名字就可以。了。  
素材文件都在roop_colab文件中   
1.jpg：换脸图片  
2.mp4：换脸视频  
out.mp4：是换脸后的视频  

点击左侧文件，找到MP4,右键可以下载！



## 解除NSFW封印


已经默认解除封印！  




## 大文件处理

如果文件比较大，直接在Colab上传会有难度，可以先传到谷歌云盘。然后挂载云盘就可以了，修改一下第五步里面图片和视频的地址就可以了。挂载谷歌盘之后也可以把输出文件直接指向云盘。

如何需要使用谷歌云盘的文件，可以点击这个脚本，挂载云盘。

from google.colab import drive  
drive.mount('/content/drive')